# 文本分类实例

step 1 导入相关的包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

step 2 加载数据

In [2]:
import pandas as pd
data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [7]:
# 删除空数据
data = data.dropna()
print(data)
print(data.iloc[0]["label"])

      label                                             review
0         1  距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较...
1         1                       商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2         1         早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3         1  宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4         1               CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风
...     ...                                                ...
7761      0  尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762      0  盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763      0  看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764      0  我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...
7765      0  说实在的我很失望，之前看了其他人的点评后觉得还可以才去的，结果让我们大跌眼镜。我想这家酒店以...

[7765 rows x 2 columns]
1


step 3 创建dataset

In [9]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        self.data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()
    
    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    
    def __len__(self):
        return len(self.data)

In [10]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


step 4 划分数据集

In [11]:
from torch.utils.data import random_split

trainset, validset = random_split(dataset, lengths=[0.9, 0.1])
len(trainset), len(validset)

(6989, 776)

In [12]:
for i in range(10):
    print(trainset[i])

('很舒服的酒店，床很软，酒店内部的布局也很有特色，而且服务不错，因为自己没说明要双床，结果到了房间后再要求跟前台换，前台态度很好，非常及时的更换了', 1)
('这是一家市政府指定招待所，分好几个区域，住的是贵宾楼，应该是宾馆中最好的区域吧。实在话，宾馆的服务员素质不错，会微笑服务，见了面会打招呼。早餐质量很一般，有点失望。但在东营，我想在东城办事的人来说，这是首选，西城的海天虽好，但离东城有二十几公里地。', 1)
('1、服务不错，安静。2、不足是设备旧了点。', 1)
('设施太陈旧了，位置离火车站近，但附近差不多条件却价格更便宜的酒店多得很！我住九层，晚上睡觉的环境都不好，比较吵，糟杂，我房间对面及左右的房间门都不关，玩通宵牌的，人多嘴杂（也没有酒店工作人员制止）！这酒店真的不太好！', 0)
('这家如家地段不算太好，所以感觉性价比不是太好，下次不去了。', 1)
('订的是豪华大床房，到那里后豪华房已经没有了，免费升级到行政大床房。过程中服务的态度还好，就是手续有点繁复，行政房需要到22层checkin除了行政房上网free，其他房间上网需要40元/小时80元/一天，免费升级到行政房上网也需要付费。之前住过豪华双人房，感觉和行政大床房没有太大差别，房间大小和装修差不多。这家宾馆最舒服的一点就是浴室比一般宾馆大。总体来说，住过这家宾馆两次，装修和房间大小包括浴室在内，在上海四星级宾馆里来说还算不错的。', 1)
('隔音太差，稍微大点声，三和板的传音效果较好。黄金周的价格是平时的一倍，明显有上当的感觉。特别是结帐时听到别人才260，我的居然是370！！！！！！！！！！！！', 0)
('到缙云也没别的地方好住,在这里住了个套房,感觉还可以,在一个小县城里有这样的地方就算不错了.房间是设备尚可,玻璃是两层的,主要是因为靠近火车站的缘故,而火车汽笛声音又很有穿透力.以后来,还必须住这里.', 1)
('房间还不错,就是电视有点高,看着有点累,前台服务还可以.', 1)
('房间太旧，卫生间是很多锈迹，房间比较大，大床不错，我喜欢。价格还可以，就是太旧了，不舒服，服务员水平差不多二十年前老国营的水平。下次会换一家住。不过据说小姐不错，俺没试。', 0)


step 5 创建dataloader

In [ ]:
import torch

tokenizer = AutoTokenizer.from_pretrained("rbt")
tokenizer("你好，世界！")

{'input_ids': [101, 872, 1962, 8024, 686, 4518, 8013, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [29]:
def collate_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])  # 文本内容
        labels.append(item[1]) # 分类标签
    
    inputs = tokenizer(text=texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

In [30]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_func)
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=collate_func)

In [ ]:
print(next(enumerate(validloader))[1])

step 6 创建模型和优化器

In [21]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("rbt", num_labels=2).to("cuda")
optimizer = Adam(model.parameters(), lr=2e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at rbt and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


step 7 训练模型以及验证

In [22]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(validset)

def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, acc: {acc}")

step 8 train

In [32]:
train()

ep: 0, global_step: 0, loss: 0.7531994581222534
ep: 0, global_step: 100, loss: 0.29347294569015503
ep: 0, global_step: 200, loss: 0.2452806681394577
ep: 0, acc: 0.8853092789649963
ep: 1, global_step: 300, loss: 0.10245198756456375
ep: 1, global_step: 400, loss: 0.48299968242645264
ep: 1, acc: 0.8891752362251282
ep: 2, global_step: 500, loss: 0.18069346249103546
ep: 2, global_step: 600, loss: 0.2787679433822632
ep: 2, acc: 0.8865979313850403


step 9 模型预测

In [34]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k : v.cuda() for k ,v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(pred)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

tensor([1], device='cuda:0')
输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！


In [36]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)
pipe(sen)

[{'label': '好评！', 'score': 0.9886162877082825}]